# GIVbacks Scripts

This Jupyter Notebook calculates GIVbacks rewards for GIVers. For more detailed information about the specs behind the distribution, check the [README](https://github.com/Giveth/givbacks-scripts).

In [2]:
import json
import requests
import pandas as pd
from web3 import Web3
from graphqlclient import GraphQLClient

*Reminder: change the variables below before running all cells.*

In [3]:
# Date Format: YYYY/MM/DD-HH:MM:SS

START_DATE = '2018/05/01-00:00:00'
END_DATE = '2021/12/31-00:00:00'

ROUND = 1

GIV_PRICE = 1
GIV_AVAILABLE = 1000
GIV_MAX_FACTOR = 0.75

## Fetching TRACE donations

First, we fetch donations made at Giveth TRACE. It's not necessary to make many data manipulations since the API already returns the data formatted.

In [33]:
BASE_TRACE_URL = 'https://feathers.beta.giveth.io/'

# TO-DO: Remove allProjects=true to fetch only verified projects
TRACE_ENDPOINT = f'{BASE_TRACE_URL}verifiedProjectsGiversReport?fromDate={START_DATE}&toDate={END_DATE}&allProjects=true'

In [6]:
response = requests.get(TRACE_ENDPOINT)

trace_data = response.json()['data']

trace_df = pd.DataFrame(trace_data)

In [7]:
trace_df = trace_df[['giverAddress', 'totalAmount']]
trace_df = trace_df.rename(columns={'totalAmount': 'totalUsd'})
trace_df['giverAddress'] = trace_df['giverAddress'].str.lower()

In [10]:
trace_df.head()

,giverAddress,totalUsd
0,0x10a84b835c5df26f2a380b3e00bcc84a66cd2d34,16488.45
1,0x839395e20bbb182fa440d08f850e6c7a8f6f0780,14942.97
2,0xf12df5ba9eadb67dda0702aaa27e813b9ef76d03,12246.66
3,0x236daa98f115caa9991a3894ae387cdc13eaad1b,8254.00
4,0x90f8bf6a479f320ead074411a4b0e7944ea8c9c1,7445.33


## Fetching Giveth donations

Then, we fetch donations made at Giveth. The GraphQL API returns all donations, consequently, we need to filter them.

In [11]:
GIVETH_ENDPOINT = 'https://mainnet.serve.giveth.io/graphql/'

In [12]:
client = GraphQLClient(GIVETH_ENDPOINT)

query = '''
{
  donations {
    valueUsd  
    createdAt
    project {
      giveBacks
    }
    user {
      walletAddress
    }
  }
}
'''

result = client.execute(query)


In [13]:
giveth_data = json.loads(result)['data']['donations']

giveth_df = pd.DataFrame(giveth_data)

In [14]:
giveth_df = giveth_df.dropna()

In [15]:
START_TIMESTAMP = pd.to_datetime(START_DATE).tz_localize('utc')
END_TIMESTAMP = pd.to_datetime(END_DATE).tz_localize('utc')

giveth_df['createdAt'] = pd.to_datetime(giveth_df['createdAt'])

giveth_df = giveth_df[
    (giveth_df['createdAt'] >= START_TIMESTAMP) & 
    (giveth_df['createdAt'] < END_TIMESTAMP)
]

In [16]:
giveth_df['giverAddress'] = giveth_df['user'].map(lambda x: x['walletAddress'].lower())
giveth_df['isVerified'] = giveth_df['project'].map(lambda x: x['giveBacks'])

In [17]:
# TO-DO: Change condition to keep only verified projects 
giveth_df = giveth_df[giveth_df['isVerified'] == False]

In [18]:
giveth_df = giveth_df[['giverAddress', 'valueUsd']]

In [19]:
giveth_df = giveth_df.groupby('giverAddress')['valueUsd'].sum().reset_index().rename(columns={'valueUsd':'totalUsd'})

In [20]:
giveth_df.head()

,giverAddress,totalUsd
0,0x001fbfe0e74adedea03e6ad3e2fc1eee1a2d5045,6.000000
1,0x0023d9ffe7d793895c5da5833d56c737832ead2b,3.300000
2,0x00d18ca9782be1caef611017c2fbc1a39779a57c,1847.998038
3,0x00df99e12315cac90f49527b07d753b7fe9faccd,1.100000
4,0x013274aa75bbc3bd7acff906e564edbbb50dce08,1.100000


## Merging donations datasets

After fetching donations, we merge both datasets and sum the total donated by GIVer.

In [21]:
donations = pd.concat([giveth_df, trace_df])

donations['giverAddress'] = donations['giverAddress'].apply(Web3.toChecksumAddress)

In [22]:
donations = donations.groupby('giverAddress').sum()

In [23]:
donations

,totalUsd
giverAddress,
0x001fbfe0E74aDedeA03e6Ad3e2Fc1eee1a2d5045,6.000000
0x0023d9ffe7D793895c5dA5833d56c737832Ead2b,3.300000
0x00d18ca9782bE1CaEF611017c2Fbc1a39779A57C,1847.998038
0x00dF99E12315Cac90f49527b07D753b7FE9fAcCd,1.100000
0x00dcc3E8378D898F7eDB0C35d69b2E38E02319ba,74.160000
...,...
0xfF7b5469f1703eB9C54297dB9f56e1d8AA4d60e5,2.200000
0xfa32aB6215Fea5e34D32a0180FF75359Bf9A1A94,2.830000
0xfc8aCF3Cbbf75Fb0e53CCeCeA657b31723BE51Fd,1.100000


## Checking how much GIV will be distributed

The amount of GIV tokens to distributed is defined below. Check the [README](https://github.com/Giveth/givbacks-scripts) for more detailed information.

In [25]:
TOTAL_RAISED = donations['totalUsd'].sum()

GIV_WORTH = GIV_PRICE * GIV_AVAILABLE

In [26]:
if TOTAL_RAISED > GIV_WORTH:
    GIV_DISTRIBUTED = GIV_AVAILABLE
else:
    GIV_DISTRIBUTED = GIV_MAX_FACTOR * TOTAL_RAISED / GIV_PRICE

## Calculating GIVbacks

With the amount of GIV, we can distribute the tokens to the GIVers proportionally to their donations.

In [27]:
donations['share'] = donations['totalUsd'] / TOTAL_RAISED

In [28]:
donations['givbacks'] = GIV_DISTRIBUTED * donations['share']

In [30]:
donations['givbacks'].sum()

1000.0

Finally, the report is generated and stored in the `reports` folder.

In [32]:
filename = f'reports/{ROUND}.json'

report = donations['givbacks'].apply(lambda x: format(x, f'.{12}f'))

report.to_json(filename, indent=4)